In [200]:
import pandas as pd
import numpy as np
import os
import pickle

## Load results 

In [201]:
with open('results_summary.dict', 'rb') as f:
    stats = pickle.load(f)

## Main tables

In [202]:
def get_latex(df):
    print(df.to_latex().replace('\$', '$'))
    # print(df.style.to_latex().replace('\$', '$'))

In [203]:
def get_results_table(results_dict, metrics, keep_methods):
    d = {}
    missing_methods = []
    for method in keep_methods:
        if method in results_dict:
            # d[method] = ['{:.4f} ({:.2f})'.format(*results_dict[method][metric]) if 'time' in metric else \
            #              '{:.4f} ({:.2f})'.format(*results_dict[method][metric]) for metric in metrics]
            d[method] = ['{:.4f}'.format(*results_dict[method][metric]) if 'time' in metric else \
                         '{:.4e}'.format(*results_dict[method][metric]) for metric in metrics]
        else:
            missing_methods.append(method)
    df = pd.DataFrame.from_dict(d, orient='index')
    df.columns = metrics
    df.index.names = ['alg']
    if len(missing_methods) > 0:
        print('missing methods: {}'.format(missing_methods))
    return df.loc[[x for x in keep_methods if x not in missing_methods]], missing_methods

In [204]:
def get_full_results_table(results_dict, exper_type, num=5):
    metrics = ['test_eval', 
               'test_eq_max', 'test_eq_mean', 
               'test_ineq_max', 'test_ineq_mean',
               'test_time']
    metrics_raw = [x.replace('test_', 'test_raw_') for x in metrics]
    metrics_renaming = ['Obj. value', 
                        'Max eq viol.', 'Mean eq viol.', 
                        'Max ineq viol.', 'Mean ineq viol.',
                        'Time']
    
    nn_methods = ['method', 'method_no_compl', 'method_no_corr', 'method_no_soft', 'method_no_corr-noTestCorr',
               'baseline_nn', 'baseline_eq_nn'] + ['method_eq_proj', 'NN_EqH_Bis', 'NN_EqD_Proj']
    opt_methods = dict([
            ('simple', ['osqp', 'qpth']), ('nonconvex', ['ipopt']), ('acopf', ['pypower']), ('qcqp', ['cvxpy']), ('dcopf', ['osqp', 'gurobi'])
    ])
    opt_methods_list = ['baseline_opt_{}'.format(x) for x in opt_methods[exper_type]]
    methods_renaming_dict = dict((
            ('method', 'DC3'),
            ('method_no_compl', 'DC3, $\neq$'),
            ('method_no_corr', 'DC3, $\not\leq$ train'),
            ('method_no_corr-noTestCorr', 'DC3, $\not\leq$ train/test'),
            ('method_no_soft', 'DC3, no soft loss'),
            ('baseline_nn-noTestCorr', 'NN'),
            ('baseline_nn', 'NN, $\leq$ test'),
            ('baseline_eq_nn-noTestCorr', 'Eq.~NN'),
            ('baseline_eq_nn', 'Eq.~NN, $\leq$ test'),
            # ('baseline_opt_osqp', 'Optimizer (OSQP)'),
            # ('baseline_opt_qpth', 'Optimizer (qpth)'),
            # ('baseline_opt_ipopt', 'Optimizer (IPOPT)'),
            # ('baseline_opt_pypower', 'Optimizer (PYPOWER)'),
            # ('baseline_opt_cvxpy', 'Optimizer (CVXPY)'),
            # ('baseline_opt_gurobi', 'Optimizer (GUROBI)'),
            ('baseline_opt_osqp', 'OSQP'),
            ('baseline_opt_qpth', 'qpth'),
            ('baseline_opt_ipopt', 'IPOPT'),
            ('baseline_opt_pypower', 'PYPOWER'),
            ('baseline_opt_cvxpy', 'CVXPY'),
            ('baseline_opt_gurobi', 'GUROBI'),
            ('method_eq_proj', 'methodEqProj'),
            ('NN_EqH_Bis', 'NN_EqH_Bis'),
            ('NN_EqD_Proj', 'NN_EqD_Proj')
        ))
    
    # df1, missing_methods = get_results_table(results_dict, metrics, nn_methods + opt_methods_list)
    
    # df2, missing_methods_2 = get_results_table(results_dict, metrics_raw, nn_methods)
    # df2.columns = metrics
    # df2.index =  ['{}-noTestCorr'.format(x) for x in df2.index]
    
    # missing_methods = missing_methods + ['{}-noTestCorr'.format(x) for x in missing_methods_2]
        
    # all_results = pd.concat([df1, df2])
    # results_ordering = opt_methods_list + ['method', 'method_no_compl', 'method_no_corr', 'method_no_corr-noTestCorr',
    #                    'method_no_soft', 'baseline_nn-noTestCorr', 'baseline_nn', 'baseline_eq_nn-noTestCorr', 
    #                    'baseline_eq_nn'] + ['method_eq_proj']
    # all_results = all_results.loc[[x for x in results_ordering if x not in missing_methods]]
    # all_results.index = [methods_renaming_dict[x] for x in all_results.index]
    # all_results.columns = metrics_renaming

    df1, missing_methods = get_results_table(results_dict, metrics, nn_methods + opt_methods_list)

    all_results = df1
    results_ordering = opt_methods_list + ['method', 'method_no_compl', 'method_no_corr',
                       'method_no_soft', 'baseline_nn', 
                       'baseline_eq_nn'] + ['method_eq_proj'] + ['NN_EqH_Bis', 'NN_EqD_Proj']
    all_results = all_results.loc[[x for x in results_ordering if x not in missing_methods]]
    all_results.index = [methods_renaming_dict[x] for x in all_results.index]
    all_results.columns = metrics_renaming
    
    return all_results

In [205]:
df_simple_5050 = get_full_results_table(stats['simple_ineq50_eq50'], exper_type='simple')
df_simple_5050

missing methods: ['method_no_compl', 'method_no_soft', 'method_no_corr-noTestCorr', 'baseline_nn', 'baseline_eq_nn', 'NN_EqH_Bis', 'NN_EqD_Proj']


,Obj. value,Max eq viol.,Mean eq viol.,Max ineq viol.,Mean ineq viol.,Time
OSQP,-1.5172e+01,8.9825e-05,4.1405e-05,2.7828e-04,2.9434e-05,0.0009
qpth,-1.5172e+01,4.5738e-15,1.2066e-15,0.0000e+00,0.0000e+00,0.0323
DC3,-1.3635e+01,1.5691e-13,5.2212e-14,0.0000e+00,0.0000e+00,0.0047
"DC3, $\not\leq$ train",-2.1928e+00,9.3901e-14,3.2716e-14,0.0000e+00,0.0000e+00,0.0003
methodEqProj,-1.4644e+01,3.5102e-14,1.0729e-14,0.0000e+00,0.0000e+00,0.0002


In [206]:
df_dcopf3970 = get_full_results_table(stats['dcopf'], exper_type='dcopf')
df_dcopf3970

missing methods: ['method', 'method_no_compl', 'method_no_corr', 'method_no_soft', 'method_no_corr-noTestCorr', 'baseline_nn', 'baseline_eq_nn', 'NN_EqH_Bis', 'NN_EqD_Proj']


,Obj. value,Max eq viol.,Mean eq viol.,Max ineq viol.,Mean ineq viol.,Time
OSQP,9.6322e+05,5.7963e-06,9.8366e-08,6.8819e-04,1.2361e-06,0.6653
GUROBI,9.5393e+05,7.9647e-09,6.6689e-12,0.0000e+00,0.0000e+00,0.1611
methodEqProj,1.0028e+06,5.3453e-08,1.9757e-11,6.6392e-04,4.0427e-08,0.0099


In [207]:
df_simple_500500 = get_full_results_table(stats['simple_ineq500_eq500'], exper_type='simple')
df_simple_500500

missing methods: ['method_no_compl', 'method_no_soft', 'method_no_corr-noTestCorr', 'baseline_nn', 'baseline_eq_nn', 'NN_EqH_Bis', 'NN_EqD_Proj', 'baseline_opt_qpth']


,Obj. value,Max eq viol.,Mean eq viol.,Max ineq viol.,Mean ineq viol.,Time
OSQP,-1.6698e+02,1.2777e-07,1.9350e-08,6.9190e-05,3.4839e-06,0.3766
DC3,2.3836e+13,6.3104e-08,1.2982e-08,2.1754e+07,2.6895e+06,0.0614
"DC3, $\not\leq$ train",-4.7228e+00,2.1850e-12,3.6344e-13,2.8088e-01,9.2045e-04,0.0051
methodEqProj,-1.6240e+02,1.0996e-13,1.9156e-14,1.7483e-03,3.4965e-06,0.0003


In [208]:
df_simple_5050_defi = get_full_results_table(stats['simple_ineq50_eq50'], exper_type='simple')
df_simple_5050_defi

missing methods: ['method_no_compl', 'method_no_soft', 'method_no_corr-noTestCorr', 'baseline_nn', 'baseline_eq_nn', 'NN_EqH_Bis', 'NN_EqD_Proj']


,Obj. value,Max eq viol.,Mean eq viol.,Max ineq viol.,Mean ineq viol.,Time
OSQP,-1.5172e+01,8.9825e-05,4.1405e-05,2.7828e-04,2.9434e-05,0.0009
qpth,-1.5172e+01,4.5738e-15,1.2066e-15,0.0000e+00,0.0000e+00,0.0323
DC3,-1.3635e+01,1.5691e-13,5.2212e-14,0.0000e+00,0.0000e+00,0.0047
"DC3, $\not\leq$ train",-2.1928e+00,9.3901e-14,3.2716e-14,0.0000e+00,0.0000e+00,0.0003
methodEqProj,-1.4644e+01,3.5102e-14,1.0729e-14,0.0000e+00,0.0000e+00,0.0002


In [209]:
# df_qcqp = get_full_results_table(stats['qcqp'], exper_type='qcqp')
# df_qcqp

In [210]:
df_simple_100100 = get_full_results_table(stats['simple_ineq100_eq100'], exper_type='simple')
df_simple_100100

missing methods: ['method_no_compl', 'method_no_soft', 'method_no_corr-noTestCorr', 'baseline_nn', 'baseline_eq_nn', 'NN_EqH_Bis', 'NN_EqD_Proj']


,Obj. value,Max eq viol.,Mean eq viol.,Max ineq viol.,Mean ineq viol.,Time
OSQP,-2.9221e+01,9.5370e-07,3.0356e-07,6.0479e-04,2.9209e-05,0.0043
qpth,-2.9221e+01,1.2091e-14,2.7528e-15,0.0000e+00,0.0000e+00,0.0846
DC3,-2.6086e+01,5.9073e-13,1.4182e-13,5.3736e-02,5.3736e-04,0.0065
"DC3, $\not\leq$ train",-6.6282e+00,2.8072e-13,8.8755e-14,0.0000e+00,0.0000e+00,0.0004
methodEqProj,-2.8777e+01,5.5381e-14,1.4542e-14,0.0000e+00,0.0000e+00,0.0002


In [211]:
df_nonconvex = get_full_results_table(stats['nonconvex'], exper_type='nonconvex')
df_nonconvex

missing methods: ['method_no_compl', 'method_no_corr', 'method_no_soft', 'method_no_corr-noTestCorr', 'NN_EqH_Bis', 'NN_EqD_Proj']


,Obj. value,Max eq viol.,Mean eq viol.,Max ineq viol.,Mean ineq viol.,Time
IPOPT,-2.2375e+01,7.7179e-15,1.8311e-15,6.2904e-08,6.2049e-09,0.1161
DC3,-2.0277e+01,4.0248e-13,9.9131e-14,1.7192e-02,1.8917e-04,0.0093
"NN, $\leq$ test",-2.0812e+01,6.3135e-01,2.3524e-01,0.0000e+00,0.0000e+00,0.0002
"Eq.~NN, $\leq$ test",-2.1740e+01,4.3185e-13,1.0484e-13,1.6263e+00,5.1761e-02,0.0094
methodEqProj,-2.1987e+01,1.4276e-14,3.4917e-15,7.8178e-04,7.8178e-06,0.0017


In [212]:
df_dcopf200 = get_full_results_table(stats['dcopf'], exper_type='dcopf')
df_dcopf200

missing methods: ['method', 'method_no_compl', 'method_no_corr', 'method_no_soft', 'method_no_corr-noTestCorr', 'baseline_nn', 'baseline_eq_nn', 'NN_EqH_Bis', 'NN_EqD_Proj']


,Obj. value,Max eq viol.,Mean eq viol.,Max ineq viol.,Mean ineq viol.,Time
OSQP,9.6322e+05,5.7963e-06,9.8366e-08,6.8819e-04,1.2361e-06,0.6653
GUROBI,9.5393e+05,7.9647e-09,6.6689e-12,0.0000e+00,0.0000e+00,0.1611
methodEqProj,1.0028e+06,5.3453e-08,1.9757e-11,6.6392e-04,4.0427e-08,0.0099


In [213]:
# df_acopf = get_full_results_table(stats['acopf'], exper_type='acopf')
# df_acopf

## Appendix tables (simple problem)

In [214]:
def get_comparison_table(all_stats, experiments, metrics, keep_methods):
    d = {}
    for method in keep_methods:
        for metric in metrics:
            d[(method, metric)] = {}
            for experiment in experiments:
                # d[(method, metric)][experiment] = '{:.2f} ({:.2f})'.format(*all_stats[experiment][method][metric])
                d[(method, metric)][experiment] = '{:.4f}'.format(*all_stats[experiment][method][metric])
        df = pd.DataFrame.from_dict(d, orient='index')           
    df.index.names = ['Alg', 'Metric']
    return df.iloc[df.index.isin(keep_methods, 'Alg')]

In [215]:
def get_simple_comparison_table(all_stats, vary_across):
    assert vary_across in ['ineq', 'eq'], 'vary_across should be in [ineq, eq]'
    
    if vary_across == 'ineq':
        experiments = ['simple_ineq10_eq50', 'simple_ineq30_eq50', 'simple_ineq50_eq50', 'simple_ineq70_eq50',
                       'simple_ineq90_eq50']
        experiments_renaming = ['10', '30', '50', '70', '90']
    else:
        experiments = ['simple_ineq50_eq10', 'simple_ineq50_eq30', 'simple_ineq50_eq50', 'simple_ineq50_eq70',
                       'simple_ineq50_eq90']
        experiments_renaming = ['10', '30', '50', '70', '90']
    
    # methods = ['method', 'method_no_compl', 'method_no_corr', 'method_no_soft', 
    #            'baseline_nn', 'baseline_eq_nn', 'baseline_opt_osqp']
    methods = ['method', 'method_no_corr', 'baseline_opt_osqp', 'method_eq_proj']
    methods_renaming_dict = dict((
            ('method', 'DC3'),
            ('method_no_compl', 'DC3, $\neq$'),
            ('method_no_corr', 'DC3, $\not\leq$ train'),
            ('method_no_corr-noTestCorr', 'DC3, $\not\leq$ train/test'),
            ('method_no_soft', 'DC3, no soft loss'),
            ('baseline_nn-noTestCorr', 'NN'),
            ('baseline_nn', 'NN, $\leq$ test'),
            ('baseline_eq_nn-noTestCorr', 'Eq.~NN'),
            ('baseline_eq_nn', 'Eq.~NN, $\leq$ test'),
            ('baseline_opt_osqp', 'Optimizers (OSQP, qpth)'),
            ('method_eq_proj', 'method_eq_proj')
        ))
    
    metrics = ['test_eval', 'test_eq_max', 'test_ineq_max']
    metrics_renaming_dict = dict((('test_eval', 'Obj. val.'), ('test_eq_max', 'Max eq.'), ('test_ineq_max', 'Max ineq.')  ))
    clean_metrics = [metrics_renaming_dict[x] for x in metrics]
    metrics_raw = [x.replace('test_', 'test_raw_') if 'time' not in x else x for x in metrics]
    
    df1 = get_comparison_table(all_stats, experiments, metrics, methods)
    df1 = df1.reset_index()
    df1['Metric'] = df1['Metric'].str.replace('_obj_val', '_eval') # make naming consistent
    df1 = df1.set_index(['Alg', 'Metric'])
    
    # df2 = get_comparison_table(all_stats, experiments, metrics_raw, methods[:-1])
    # df2.columns = experiments
    # df2 = df2.reset_index()
    # df2['Alg'] = ['{}-noTestCorr'.format(x) for x in df2['Alg']]
    # df2['Metric'] = df2['Metric'].str.replace('raw_', '')  # make naming consistent
    # df2 = df2.set_index(['Alg', 'Metric'])
    
    # all_results = pd.concat([df1, df2])
    all_results = df1
    # results_ordering = ['baseline_opt_osqp', 'method', 'method_no_compl', 'method_no_corr', 'method_no_corr-noTestCorr',
    #                    'method_no_soft', 'baseline_nn-noTestCorr', 'baseline_nn', 'baseline_eq_nn-noTestCorr', 
    #                    'baseline_eq_nn'] + ['method_eq_proj']
    results_ordering = ['baseline_opt_osqp', 'method', 'method_no_corr', 'method_eq_proj']
    clean_results_ordering = [methods_renaming_dict[x] for x in results_ordering]
    all_results = all_results.iloc[all_results.index.isin(results_ordering, 'Alg')]
    all_results = all_results.reindex(pd.MultiIndex.from_product([results_ordering, metrics], names=['Alg', 'Metric']))
    all_results = all_results.reset_index()
    all_results['Alg'] = [methods_renaming_dict[x] for x in all_results['Alg']]
    all_results['Metric'] = [metrics_renaming_dict[x] for x in all_results['Metric']]
    all_results = all_results.set_index(['Alg', 'Metric'])
    all_results.columns = experiments_renaming
    all_results = all_results.reindex(pd.MultiIndex.from_product([clean_results_ordering, clean_metrics])) # remove names
    
    return all_results

In [216]:
df_simple_varyeq = get_simple_comparison_table(stats, 'eq')
df_simple_varyeq

10        30        50        70  \
Optimizers (OSQP, qpth) Obj. val.  -27.4169  -23.1372  -15.1717  -15.1843   
                        Max eq.      0.0000    0.0000    0.0001    0.0000   
                        Max ineq.    0.0000    0.0000    0.0003    0.0000   
DC3                     Obj. val.  -25.8963  -19.9340  -13.6346  -14.1132   
                        Max eq.      0.0000    0.0000    0.0000    0.0000   
                        Max ineq.    0.0000    0.0000    0.0000    0.0000   
DC3, $\not\leq$ train   Obj. val.  -25.9344  -20.1796   -2.1928  -13.9190   
                        Max eq.      0.0000    0.0000    0.0000    0.0000   
                        Max ineq.    0.0000    0.0000    0.0000    0.0000   
method_eq_proj          Obj. val.  -26.8866  -22.6336  -14.6441  -14.9955   
                        Max eq.      0.0000    0.0000    0.0000    0.0000   
                        Max ineq.    0.0000    0.0000    0.0000    0.0000   

                                        90  
Optimizers (OSQP, qpth) Obj. val.  -4.1482  
                        Max eq.     0.0003  
                        Max ineq.   0.0000  
DC3                     Obj. val.  -4.1143  
                        Max eq.     0.0000  
                        Max ineq.   0.0000  
DC3, $\not\leq$ train   Obj. val.  -4.1231  
                        Max eq.     0.0000  
                        Max ineq.   0.0000  
method_eq_proj          Obj. val.  -4.1476  
                        Max eq.     0.0000  
                        Max ineq.   0.0000

In [217]:
df_simple_varyineq = get_simple_comparison_table(stats, 'ineq')
df_simple_varyineq

10        30        50        70  \
Optimizers (OSQP, qpth) Obj. val.  -17.4758  -16.4697  -15.1717  -14.7489   
                        Max eq.      0.0003    0.0003    0.0001    0.0000   
                        Max ineq.    0.0002    0.0004    0.0003    0.0001   
DC3                     Obj. val.  -15.8952  -14.3480  -13.6346  -11.1910   
                        Max eq.      0.0000    0.0000    0.0000    0.0000   
                        Max ineq.    0.0000    0.0000    0.0000    0.0000   
DC3, $\not\leq$ train   Obj. val.  -15.6371  -14.2110   -2.1928  -11.2128   
                        Max eq.      0.0000    0.0000    0.0000    0.0000   
                        Max ineq.    0.0000    0.0000    0.0000    0.0000   
method_eq_proj          Obj. val.  -17.2931  -16.1412  -14.6441  -14.0622   
                        Max eq.      0.0000    0.0000    0.0000    0.0000   
                        Max ineq.    0.0000    0.0000    0.0000    0.0000   

                                         90  
Optimizers (OSQP, qpth) Obj. val.  -14.4268  
                        Max eq.      0.0000  
                        Max ineq.    0.0001  
DC3                     Obj. val.  -11.8657  
                        Max eq.      0.0000  
                        Max ineq.    0.0000  
DC3, $\not\leq$ train   Obj. val.  -11.9895  
                        Max eq.      0.0000  
                        Max ineq.    0.0000  
method_eq_proj          Obj. val.  -13.6941  
                        Max eq.      0.0000  
                        Max ineq.    0.0000

## LaTeX 

In [218]:
get_latex(df_dcopf200)

\begin{tabular}{lllllll}
\toprule
{} &  Obj. value & Max eq viol. & Mean eq viol. & Max ineq viol. & Mean ineq viol. &    Time \\
\midrule
OSQP         &  9.6322e+05 &   5.7963e-06 &    9.8366e-08 &     6.8819e-04 &      1.2361e-06 &  0.6653 \\
GUROBI       &  9.5393e+05 &   7.9647e-09 &    6.6689e-12 &     0.0000e+00 &      0.0000e+00 &  0.1611 \\
methodEqProj &  1.0028e+06 &   5.3453e-08 &    1.9757e-11 &     6.6392e-04 &      4.0427e-08 &  0.0099 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_1554898/3399575891.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex().replace('\$', '$'))


In [219]:
get_latex(df_simple_5050)

\begin{tabular}{lllllll}
\toprule
{} &   Obj. value & Max eq viol. & Mean eq viol. & Max ineq viol. & Mean ineq viol. &    Time \\
\midrule
OSQP                  &  -1.5172e+01 &   8.9825e-05 &    4.1405e-05 &     2.7828e-04 &      2.9434e-05 &  0.0009 \\
qpth                  &  -1.5172e+01 &   4.5738e-15 &    1.2066e-15 &     0.0000e+00 &      0.0000e+00 &  0.0323 \\
DC3                   &  -1.3635e+01 &   1.5691e-13 &    5.2212e-14 &     0.0000e+00 &      0.0000e+00 &  0.0047 \\
DC3, $\textbackslash not\textbackslash leq$ train &  -2.1928e+00 &   9.3901e-14 &    3.2716e-14 &     0.0000e+00 &      0.0000e+00 &  0.0003 \\
methodEqProj          &  -1.4644e+01 &   3.5102e-14 &    1.0729e-14 &     0.0000e+00 &      0.0000e+00 &  0.0002 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_1554898/3399575891.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex().replace('\$', '$'))


In [220]:
get_latex(df_nonconvex)

\begin{tabular}{lllllll}
\toprule
{} &   Obj. value & Max eq viol. & Mean eq viol. & Max ineq viol. & Mean ineq viol. &    Time \\
\midrule
IPOPT               &  -2.2375e+01 &   7.7179e-15 &    1.8311e-15 &     6.2904e-08 &      6.2049e-09 &  0.1161 \\
DC3                 &  -2.0277e+01 &   4.0248e-13 &    9.9131e-14 &     1.7192e-02 &      1.8917e-04 &  0.0093 \\
NN, $\textbackslash leq$ test     &  -2.0812e+01 &   6.3135e-01 &    2.3524e-01 &     0.0000e+00 &      0.0000e+00 &  0.0002 \\
Eq.\textasciitilde NN, $\textbackslash leq$ test &  -2.1740e+01 &   4.3185e-13 &    1.0484e-13 &     1.6263e+00 &      5.1761e-02 &  0.0094 \\
methodEqProj        &  -2.1987e+01 &   1.4276e-14 &    3.4917e-15 &     7.8178e-04 &      7.8178e-06 &  0.0017 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_1554898/3399575891.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex().replace('\$', '$'))


In [221]:
# get_latex(df_acopf)

In [226]:
get_latex(df_simple_varyineq)

\begin{tabular}{lllllll}
\toprule
               &           &        10 &        30 &        50 &        70 &        90 \\
\midrule
Optimizers (OSQP, qpth) & Obj. val. &  -17.4758 &  -16.4697 &  -15.1717 &  -14.7489 &  -14.4268 \\
               & Max eq. &    0.0003 &    0.0003 &    0.0001 &    0.0000 &    0.0000 \\
               & Max ineq. &    0.0002 &    0.0004 &    0.0003 &    0.0001 &    0.0001 \\
DC3 & Obj. val. &  -15.8952 &  -14.3480 &  -13.6346 &  -11.1910 &  -11.8657 \\
               & Max eq. &    0.0000 &    0.0000 &    0.0000 &    0.0000 &    0.0000 \\
               & Max ineq. &    0.0000 &    0.0000 &    0.0000 &    0.0000 &    0.0000 \\
DC3, $\textbackslash not\textbackslash leq$ train & Obj. val. &  -15.6371 &  -14.2110 &   -2.1928 &  -11.2128 &  -11.9895 \\
               & Max eq. &    0.0000 &    0.0000 &    0.0000 &    0.0000 &    0.0000 \\
               & Max ineq. &    0.0000 &    0.0000 &    0.0000 &    0.0000 &    0.0000 \\
method\_eq\_proj & Obj. val. &

/tmp/ipykernel_1554898/3399575891.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex().replace('\$', '$'))


In [222]:
get_latex(df_simple_varyeq)

\begin{tabular}{lllllll}
\toprule
               &           &        10 &        30 &        50 &        70 &       90 \\
\midrule
Optimizers (OSQP, qpth) & Obj. val. &  -27.4169 &  -23.1372 &  -15.1717 &  -15.1843 &  -4.1482 \\
               & Max eq. &    0.0000 &    0.0000 &    0.0001 &    0.0000 &   0.0003 \\
               & Max ineq. &    0.0000 &    0.0000 &    0.0003 &    0.0000 &   0.0000 \\
DC3 & Obj. val. &  -25.8963 &  -19.9340 &  -13.6346 &  -14.1132 &  -4.1143 \\
               & Max eq. &    0.0000 &    0.0000 &    0.0000 &    0.0000 &   0.0000 \\
               & Max ineq. &    0.0000 &    0.0000 &    0.0000 &    0.0000 &   0.0000 \\
DC3, $\textbackslash not\textbackslash leq$ train & Obj. val. &  -25.9344 &  -20.1796 &   -2.1928 &  -13.9190 &  -4.1231 \\
               & Max eq. &    0.0000 &    0.0000 &    0.0000 &    0.0000 &   0.0000 \\
               & Max ineq. &    0.0000 &    0.0000 &    0.0000 &    0.0000 &   0.0000 \\
method\_eq\_proj & Obj. val. &  -26.8866

/tmp/ipykernel_1554898/3399575891.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex().replace('\$', '$'))


In [223]:
get_latex(df_dcopf3970)

\begin{tabular}{lllllll}
\toprule
{} &  Obj. value & Max eq viol. & Mean eq viol. & Max ineq viol. & Mean ineq viol. &    Time \\
\midrule
OSQP         &  9.6322e+05 &   5.7963e-06 &    9.8366e-08 &     6.8819e-04 &      1.2361e-06 &  0.6653 \\
GUROBI       &  9.5393e+05 &   7.9647e-09 &    6.6689e-12 &     0.0000e+00 &      0.0000e+00 &  0.1611 \\
methodEqProj &  1.0028e+06 &   5.3453e-08 &    1.9757e-11 &     6.6392e-04 &      4.0427e-08 &  0.0099 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_1554898/3399575891.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex().replace('\$', '$'))


In [224]:
get_latex(df_simple_5050_defi)

\begin{tabular}{lllllll}
\toprule
{} &   Obj. value & Max eq viol. & Mean eq viol. & Max ineq viol. & Mean ineq viol. &    Time \\
\midrule
OSQP                  &  -1.5172e+01 &   8.9825e-05 &    4.1405e-05 &     2.7828e-04 &      2.9434e-05 &  0.0009 \\
qpth                  &  -1.5172e+01 &   4.5738e-15 &    1.2066e-15 &     0.0000e+00 &      0.0000e+00 &  0.0323 \\
DC3                   &  -1.3635e+01 &   1.5691e-13 &    5.2212e-14 &     0.0000e+00 &      0.0000e+00 &  0.0047 \\
DC3, $\textbackslash not\textbackslash leq$ train &  -2.1928e+00 &   9.3901e-14 &    3.2716e-14 &     0.0000e+00 &      0.0000e+00 &  0.0003 \\
methodEqProj          &  -1.4644e+01 &   3.5102e-14 &    1.0729e-14 &     0.0000e+00 &      0.0000e+00 &  0.0002 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_1554898/3399575891.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex().replace('\$', '$'))


In [225]:
get_latex(df_simple_100100)

\begin{tabular}{lllllll}
\toprule
{} &   Obj. value & Max eq viol. & Mean eq viol. & Max ineq viol. & Mean ineq viol. &    Time \\
\midrule
OSQP                  &  -2.9221e+01 &   9.5370e-07 &    3.0356e-07 &     6.0479e-04 &      2.9209e-05 &  0.0043 \\
qpth                  &  -2.9221e+01 &   1.2091e-14 &    2.7528e-15 &     0.0000e+00 &      0.0000e+00 &  0.0846 \\
DC3                   &  -2.6086e+01 &   5.9073e-13 &    1.4182e-13 &     5.3736e-02 &      5.3736e-04 &  0.0065 \\
DC3, $\textbackslash not\textbackslash leq$ train &  -6.6282e+00 &   2.8072e-13 &    8.8755e-14 &     0.0000e+00 &      0.0000e+00 &  0.0004 \\
methodEqProj          &  -2.8777e+01 &   5.5381e-14 &    1.4542e-14 &     0.0000e+00 &      0.0000e+00 &  0.0002 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_1554898/3399575891.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex().replace('\$', '$'))
